# Feature selection

### What is feature selection?

Feature selection is a method of selecting features from your feature set to be used for modeling. It draws from a set of existing features, so it's different than feature engineering because it doesn't create new features. The goal of feature selection is to improve your model's performance. Perhaps your existing feature set is much too large, or some of the features you're working with are unnecessary.

There are different ways you can perform feature selection. It's possible to do it in an automated way. Scikit-learn has several methods for automated feature selection, such as choosing a variance threshold and using univariate statistical tests, but we won't cover those here. 

Feature selection can help to get rid of noise in your model. Perhaps you have redundant features, like latitude and longitude, city, and state. Or maybe you have features that are strongly statistically correlated, which breaks the assumptions of certain models and thus impacts model performance. If your feature set is large, it may be beneficial to use dimensionality reduction to combine and reduce the number of features in your dataset in a way that also reduces the overall variance.

### Removing features manually

One of the easiest ways to determine if a feature is unnecessary is if it is redundant in some way. For example, if it exists in another form as another feature—sometimes, when you create features through feature engineering, you end up duplicating existing features in some way. 

For example, if your dataset contains repeated information in its feature set, it's unlikely you'll need to use each feature for modeling. You may see columns related to city, state, latitude, and longitude in the same dataset, such as in the `volunteer` dataset:

In [ ]:
import pandas as pd

dir_string = "../../datasets/"

volunteer = ____(dir_string + "volunteer.csv")
volunteer.____

The `volunteer` dataset contains a lot of different information related to the location of the volunteer opportunity: `Latitude`, `Longitude`, `locality`, `region`, and `postalcode`.

Dropping columns is as simple as using pandas' `drop()` method, which we learned about previously, but it's important to remember here. Specifying `axis=1` ensures that we drop entire columns:

In [ ]:
volunteer_subset = volunteer.____(["Latitude", "Longitude", "locality", "postalcode"], ____)

volunteer_subset.____

Perhaps, for your modeling task, you only need the high-level state information in the `region` field, which we didn't drop. 

Another situation where duplicate features occur is through feature engineering. In an earlier example, we calculated a number of aggregate statistics on running times. The columns we had were times from 5 separate runs as well as mean, median, total, and fastest time. We could likely drop the values that generated the aggregate statistic. Feature selection, like most aspects of the machine learning pipeline, is very dependent on the model you're trying to build.

### Removing correlated features

Another clear situation in which you'd want to drop features is when they are statistically correlated, meaning they move together directionally. Linear models in particular assume that features are independent of each other, and if features are strongly correlated, that could introduce bias into your model. 

We'll use Pearson's correlation coefficient to check a feature set for correlation. The Pearson correlation coefficient is a measure of this directionality: a score closer to 1 between pairs of features means that they move together in the same direction more strongly, a score closer to 0 means features are not correlated, and a score close to -1 means they are more strongly negatively correlated, meaning one feature increases in value while the other decreases. 

Let's take a look at this simple dataset:

In [ ]:
corr_dict = {"col1": [1, 2, 3, 4], 
             "col2": [0.23, 3, 8, -9], 
             "col3": [10, 20, 30, 40], 
             "col4": [0.3, 0.2, 90.6, 0.1]}

corr_df = pd.DataFrame(corr_dict)

We can easily check correlation in Pandas using the `corr()` method, which outputs features and their measures of correlation:

In [ ]:
corr_df.____

As you can see, every column is obviously perfectly correlated with itself. However, we can also see that colunms 1 and 3 have a perfect correlation as well! In the toy dataset above, you can see that column 3 is simply column 1 multipled by 10, so this makes sense. We'd definitely want to remove one of those columns.

Correlation isn't usually as clear-cut as this. You can also see that columns 2 and 4 have a relatively high correlation at around 0.7. Since there are only four features in this dataset, perhaps we wouldn't remove one of those columns. This is where iteration is important - you might be able to pick the perfect feature set on the first try, or you might need to experiment with a few different configurations.

### Your turn!

Let's take a look at the `wine` dataset, which is made up of continuous, numerical features of various characteristics of wine. Read in the file and print out the `head()`:

In [ ]:
wine = ____(dir_string + "wine_types.csv")

wine.____

Let's use a subset of the `wine` dataset so we can see the output of `corr()` more easily:

In [ ]:
wine_subset = wine[["Flavanoids", "Total phenols", "Malic acid", "OD280/OD315 of diluted wines", "Hue"]]

wine_subset.____

Taking a close look at the `Flavanoids` feature after correlation, you can see that it's strongly positively correlated with both `Total phenols` and `OD280/OD315 of diluted wines`. Let's drop it from that subset, and then look at the correlation again:

In [ ]:
wine_subset = wine_subset.____

wine_subset.____

Another feature you might consider dropping is `OD280/OD315 of diluted wines`, which is strongly positively correlated with `Total Phenols` and moderately positively correlated with `Hue`. This is a situation where you'd want to iterate on your chosen features and see how they impact your model's performance.

### Dimensionality reduction for feature selection

A less manual way of reducing the size of your feature set is through dimensionality reduction. Dimensionality reduction is a form of unsupervised learning that transforms your data in a way that shrinks the number of features in your feature space. 

The method of dimensionality reduction we'll cover is principal component analysis, or PCA. PCA uses a linear transformation to project features into a space where they are completely uncorrelated. While the feature space is reduced, the variance is captured in a meaningful way by combining features into components. PCA captures, in each component, as much of the variance in the dataset as possible. In terms of feature selection, it can be a useful method when you have a large number of features and no strong candidates for elimination.

Transforming a dataset through PCA is relatively straightforward in scikit-learn. Let's apply PCA to the `wine` dataset. 

First, we'll set up our data for modeling by removing the label column, `Type`, from the `wine` dataset:

In [ ]:
wine_X = wine.____(["Type"], ____)

Next, we'll apply `PCA` to `wine_X` using `fit_transform()`, which transforms the data into components:

In [ ]:
from sklearn.decomposition import PCA

pca = ____
transformed_X = pca.____

By default, PCA in scikit-learn keeps the number of components equal to the number of input features. If we print out the `explained_variance_ratio_`, we can see, by component, the percentage of variance explained by that component:

In [ ]:
pca.____

You can see that much of the variance is explained by the first component here—around 99%—so it's likely that we could drop those components that don't explain much variance.

There are a couple of things to note regarding PCA. The first is that it can be very difficult to interpret PCA components beyond which components explain the most variance. PCA is more of a black box method than other methods of dimensionality reduction. The other thing to note is that PCA is a good step to do at the end of your preprocessing journey, because of the way the data gets transformed and reshaped. It can be difficult to do much feature work post-PCA, other than eliminating components that aren't useful in explaining variance.

### Your turn: training a model using PCA

Let's take a look at how `knn` performs using PCA components as the `X` data. First, let's create our `y` labels out of the `Type` column as well as split the PCA-transformed data. We can simply pass in the `transformed_X` data we created in the previous step to `train_test_split()` to create training and test data. 

In [ ]:
from sklearn.model_selection import train_test_split

y = ____
X_pca_train, X_pca_test, y_pca_train, y_pca_test = ____

Next, we'll create the `knn` classifier, fit it to the training data, and score it on the test data:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_pca = ____

Let's fit the model to the data and print out the score:

In [ ]:
knn_pca.____

knn_pca.____

Some questions to think about: What other improvements could you make to the `wine` dataset? Would you use PCA? Why or why not?